<a href="https://colab.research.google.com/github/JhonFiUNFV/python_prep/blob/master/05_address_matching_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clonamos el repositorio para obtener los dataSet

In [1]:
!git clone https://github.com/joanby/tensorflow.git

Cloning into 'tensorflow'...
remote: Enumerating objects: 60398, done.
remote: Total 60398 (delta 0), reused 0 (delta 0), pack-reused 60398
Receiving objects: 100% (60398/60398), 443.85 MiB | 23.07 MiB/s, done.
Resolving deltas: 100% (131/131), done.
Checking out files: 100% (60276/60276), done.


# Damos acceso a nuestro Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Test it

In [3]:
!ls '/content/drive/My Drive' 

'2. Intermedio'
 Canva
'CFA - LEVEL II - TANDEM FINANCE'
 Classroom
'Colab Notebooks'
 constancia_empleo3.pdf
 Curso_Proyectos_InviertePE
 Data
'Data (1).xlsx'
 Data.gsheet
 Data.xlsx
 GRUPOS_PY.gsheet
'informe5 (1).gsheet'
 informe5.gsheet
'Investigación UX - JHON VIDAL FIGUEROA CÉSPEDES.gsheet'
'JHON VIDAL FIGUEROA CÉSPEDES.pdf'
 LeWagon
 Master.xlsx
 ModuloII_FundamentosdeMachineLearning
 ModuloIV_ManejodeBasedeDatos
 ProgramadeEspecializacionenDataScience
 PROYECTO_CONSTRUCCION.docx
'PROYECTO GRUPO 01.pptx'
 python-ml
'QUANT 2021'
 rar
'SEMINARIO DE ADM DE RIESGO EN BANCA'
'Sesión 2-20220320T150758Z-001.zip'
 tensorflow
'Teoria Económica y Finanzas'
'TESIS JHON FIGUEROA'
 Worksheet.xlsx


# Google colab tools

In [4]:
from google.colab import files # Para manejar los archivos y, por ejemplo, exportar a su navegador
import glob # Para manejar los archivos y, por ejemplo, exportar a su navegador
from google.colab import drive # Montar tu Google drive

##Especificando la versión de TensorFlow

Ejecutando "importar tensorflow" importará la versión por defecto (actualmente 2.x). Puedes usar la 1.x ejecutando una celda con la "versión mágica de tensorflow" **antes de ejecutar "importar tensorflow".

### Si no funciona hacer el pip install


In [5]:
#!pip install tensorflow==1.14
%tensorflow_version 1.x

TensorFlow 1.x selected.


# Importar Tensorflow

In [6]:
import tensorflow as tf
print(tf.__version__)
import matplotlib.pyplot as plt

1.15.2


In [7]:
session = tf.Session()

# Correspondencia en direcciones postales

In [8]:
import random
import string
import numpy as np

In [9]:
n = 10
street_names = ["diagon", "elm", "abbey", "gran", "python"]
street_type = ["callejon", "calle", "carrera", "via", "avenida"]
street_zips = [random.randint(20000, 29999) for i in range(5)]
numbers = [random.randint(1,999) for i in range(n)]

In [10]:
streets = [random.choice(street_names) for i in range(n)]
street_prefs = [random.choice(street_type) for i in range(n)]
zips = [random.choice(street_zips) for i in range(n)]
full_streets = [x + " " + y + " " + str(z) for x,y,z in zip(street_prefs, streets, numbers)]
reference_data = [list(x) for x in zip(full_streets, zips)]

In [11]:
reference_data

[['avenida elm 798', 22833],
 ['avenida python 501', 28436],
 ['callejon python 784', 26803],
 ['callejon diagon 470', 20507],
 ['calle python 477', 20507],
 ['carrera elm 560', 23633],
 ['callejon python 225', 20507],
 ['avenida diagon 408', 23633],
 ['callejon diagon 948', 20507],
 ['carrera elm 122', 26803]]

In [12]:
def create_typo(s, prob=0.75):
    if random.uniform(0,1)<0.75:
        rand_idx = random.choice(range(len(s)))
        s_list = list(s)
        s_list[rand_idx] = random.choice(string.ascii_lowercase)
        s = ''.join(s_list)
    return s

In [13]:
typo_streets = [create_typo(x) for x in streets]

In [14]:
typo_full_streets = [x+" "+y+" "+str(z) for x,y,z in zip(street_prefs, typo_streets, numbers)]
test_data = [list(x) for x in zip(typo_full_streets, zips)]

In [15]:
test_data

[['avenida glm 798', 22833],
 ['avenida python 501', 28436],
 ['callejon yython 784', 26803],
 ['callejon dlagon 470', 20507],
 ['calle vython 477', 20507],
 ['carrera evm 560', 23633],
 ['callejon pytlon 225', 20507],
 ['avenida diagof 408', 23633],
 ['callejon xiagon 948', 20507],
 ['carrera elj 122', 26803]]

In [16]:
session = tf.Session()

In [17]:
test_address = tf.sparse_placeholder(dtype = tf.string)
test_zip = tf.placeholder(shape = [None, 1], dtype=tf.float32)

ref_address = tf.sparse_placeholder(dtype = tf.string)
ref_zip = tf.placeholder(shape=[None, n], dtype=tf.float32 )

In [18]:
zip_dist = tf.square(tf.subtract(ref_zip, test_zip))
address_dist = tf.edit_distance(test_address, ref_address, normalize=True)

- $S(x,y) = 0$ si $x$ e $y$ son totalmente diferentes (no se parecen en nada)
- $S(x,x) = 1$, ya que todo objeto es similar (si no igual) a si mismo.
- $S(x,y) = \frac{D - d(x,y)}{D-d}$ donde $D$ es la mayor distancia entre dos objetos posibles, y $d$ es la menor.

In [19]:
zip_max = tf.gather(tf.squeeze(zip_dist), tf.argmax(zip_dist, 1))
zip_min = tf.gather(tf.squeeze(zip_dist), tf.argmin(zip_dist, 1))
zip_sim = tf.divide(tf.subtract(zip_max, zip_dist), tf.subtract(zip_max, zip_min))

In [20]:
address_sim = tf.subtract(1.0, address_dist)

$$S(x,y) = \sum_{i=1}^k w_iS_k(x,y):\quad \sum_{i=1}^kw_i = 1$$

In [21]:
address_wi = 0.5
zip_wi = 1.0 -address_wi

In [22]:
weighted_sim = tf.add(tf.transpose(tf.multiply(address_wi, address_sim)),tf.multiply(zip_wi, zip_sim))

In [23]:
top_match_idx = tf.argmax(weighted_sim, 1)

In [24]:
def sparse_from_word_vector(word_vector):
    num_words = len(word_vector)
    idx = [[xi, 0, yi] for xi, x in enumerate(word_vector) for yi, y in enumerate(x)]
    chars = list(''.join(word_vector))
    return tf.SparseTensorValue(idx, chars, [num_words,1,1])

In [25]:
reference_address = [x[0] for x in reference_data]
reference_zips = np.array([[x[1] for x in reference_data]])

In [26]:
sparse_ref_set = sparse_from_word_vector(reference_address)

In [27]:
for i in range(n):
    test_address_entry = test_data[i][0]
    test_zip_entry = [[test_data[i][1]]]
    
    test_address_rep = [test_address_entry]*n
    sparse_test_set = sparse_from_word_vector(test_address_rep)
    
    feed_dict = {test_address: sparse_test_set,
                test_zip: test_zip_entry,
                ref_address: sparse_ref_set,
                ref_zip: reference_zips}
    
    best_match = session.run(top_match_idx, feed_dict=feed_dict)
    best_address = reference_address[best_match[0]]
    [best_zip] = reference_zips[0][best_match]
    [[test_zip_aux]] = test_zip_entry
    
    print("Dirección original: "+str(test_address_entry)+ ", "+str(test_zip_aux))
    print("Dirección corregida: "+str(best_address)+", "+str(best_zip)+"\n")

Dirección original: avenida glm 798, 22833
Dirección corregida: avenida elm 798, 22833

Dirección original: avenida python 501, 28436
Dirección corregida: avenida python 501, 28436

Dirección original: callejon yython 784, 26803
Dirección corregida: callejon python 784, 26803

Dirección original: callejon dlagon 470, 20507
Dirección corregida: callejon diagon 470, 20507

Dirección original: calle vython 477, 20507
Dirección corregida: calle python 477, 20507

Dirección original: carrera evm 560, 23633
Dirección corregida: carrera elm 560, 23633

Dirección original: callejon pytlon 225, 20507
Dirección corregida: callejon python 225, 20507

Dirección original: avenida diagof 408, 23633
Dirección corregida: avenida diagon 408, 23633

Dirección original: callejon xiagon 948, 20507
Dirección corregida: callejon diagon 948, 20507

Dirección original: carrera elj 122, 26803
Dirección corregida: carrera elm 122, 26803

